# 3D Lithospheric Model Rotational

romain.beucher@unimelb.edu.au

The Model is reproducing the setup of Mondy et al, 2018

Mondy, Luke & Rey, Patrice & Duclaux, Guillaume & Moresi, Louis. (2018). The role of asthenospheric flow during rift propagation and breakup. Geology. 46. 103-106. 10.1130/G39674.1. 


The following Model has been tested on NCI Raijin and Pawsey Magnus with the following configurations:

    - configuration A: (128 x 128 x 64 nodes) 128 CPUS
    - configuration B: (256 x 256 x 96 nodes) 256 CPUS
    
The user should remove visualisation from the python script before running the model on raijin.

In [1]:
import UWGeodynamics as GEO

loaded rc file /opt/UWGeodynamics/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
u = GEO.UnitRegistry

In [3]:
characteristic_time = 1 * u.second
characteristic_length = 1 * u.kilometer
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1603.15 * u.degK
characteristic_viscosity = 1e22 * u.pascal * u.second

KL = characteristic_length
Kt = characteristic_time
KM = characteristic_viscosity * KL * Kt
KT = (baseModelTemp - surfaceTemp)

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT

In [4]:
Model = GEO.Model(elementRes=(32, 32, 16), 
                  minCoord=(0. * u.kilometer, 0. * u.kilometer, -160. * u.kilometer), 
                  maxCoord=(500. * u.kilometer, 500. * u.kilometer, 20. * u.kilometer), 
                  gravity=(0.0, 0.0, -9.81 * u.meter / u.second**2))

## Global definitions

In [5]:
Model.outputDir="3D_Rift"

In [6]:
Model.maxViscosity  = 5e23 * u.pascal * u.second
Model.minViscosity  = 1e19 * u.pascal * u.second
Model.stressLimiter = 300. * u.megapascal

In [7]:
Model.diffusivity = 1e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

## Define Materials

The model has initially 4 materials (air, crust, mantle lithosphere and mantle). We add a fifth material for the sediment. Sediment will only appear if surface processes are turned on...(and if there is sedimentation of course)

In [8]:
air = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=0 * u.kilometer))
uppercrust = Model.add_material(name="UpperCrust", shape=GEO.shapes.Layer3D(top=air.bottom, bottom=-20 * u.kilometer))
lowercrust = Model.add_material(name="LowerCrust", shape=GEO.shapes.Layer3D(top=uppercrust.bottom, bottom=-40 * u.kilometer))
mantle = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=lowercrust.bottom, bottom=Model.bottom))

### Material specific definitions


In [9]:
air.density = 1. * u.kg / u.m**3
uppercrust.density  = GEO.LinearDensity(reference_density=2800. * u.kilogram / u.metre**3)
lowercrust.density  = GEO.LinearDensity(reference_density=2900. * u.kilogram / u.metre**3)
mantle.density  = GEO.LinearDensity(reference_density=3300. * u.kilogram / u.metre**3)

In [10]:
uppercrust.radiogenicHeatProd = 1.2 * u.microwatt / u.meter**3
lowercrust.radiogenicHeatProd = 0.6 * u.microwatt / u.meter**3

### Viscous Rheologies

In [11]:
rh = GEO.ViscousCreepRegistry()

In [12]:
air.viscosity                = 1e19 * u.pascal * u.second
uppercrust.viscosity         = rh.Wet_Quartz_Dislocation_Paterson_and_Luan_1990
lowercrust.viscosity         = rh.Dry_Mafic_Granulite_Dislocation_Wang_et_al_2012
mantle.viscosity             = rh.Wet_Olivine_Dislocation_Hirth_and_Kohlstedt_2003

### Plasticities

We use a Drucker-Prager yield criterion with frictional weakening.

In [13]:
pl = GEO.PlasticityRegistry()

In [14]:
uppercrust.plasticity = pl.Rey_and_Muller_2010_UpperCrust
lowercrust.plasticity = pl.Rey_and_Muller_2010_LowerCrust
mantle.plasticity = pl.Rey_and_Muller_2010_LithosphericMantle

## Passive Tracers

We create 2 sets of passive tracers to track the surface and the moho.

In [16]:
import numpy as np

xp = np.linspace(GEO.nd(Model.minCoord[0]), GEO.nd(Model.maxCoord[0]), 100)
yp = np.linspace(GEO.nd(Model.minCoord[1]), GEO.nd(Model.maxCoord[1]), 100)

xp, yp = np.meshgrid(xp, yp)
xp = xp.flatten()
yp = yp.flatten()
zp = np.zeros(xp.shape)

surface_tracers = Model.add_passive_tracers(name="Surface", vertices=[xp, yp, zp])
moho_tracers = Model.add_passive_tracers(name="Moho", vertices=[xp, yp, zp+GEO.nd(mantle.top)])

## Temperature Boundary Condition

Temperature is 293.15K at the top and 1603.15K at the bottom. Temperature is constant in the mantle and the air layers. 

In [17]:
Model.set_temperatureBCs(top=293.15 * u.degK, 
                         bottom=1603.15 * u.degK, 
                         materials=[(air, 293.15 * u.degK )])

## Velocity Boundary Conditions

We pull on the left and right side. The back and front wall are freeslip. We use a pseudo isostatic support condition at the bottom.

In [18]:
velFunc = -Model.y / GEO.nd(Model.maxCoord[1]) * GEO.nd(2.0 * u.centimeter / u.year) + GEO.nd(2.5 * u.centimetre / u.year)

In [19]:
Model.set_velocityBCs(left=[-1.0 * velFunc, None, None],
                      right=[velFunc, None, None],
                      back=[None, 0., None],
                      front=[None, 0., None],
                      bottom=GEO.LecodeIsostasy(reference_mat=mantle,
                                                average=True))

## Initial Plastic Strain

An ellipsoidal shape with random damage is used to seed plastic deformation.

In [20]:
import numpy as np

def gaussian(xx, centre, width):
    return ( np.exp( -(xx - centre)**2 / width**2 ))

maxDamage = 0.1
Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(20.0 * u.kilometer))
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,2], GEO.nd(-35. * u.kilometer) , GEO.nd(20.0 * u.kilometer))

In [21]:
import underworld.function as fn

def post_hook():
    coords = fn.input()
    zz = coords[0] / (GEO.nd(Model.maxCoord[0]) - GEO.nd(Model.minCoord[0]))
    fact = fn.math.pow(fn.math.tanh(zz*20.0) + fn.math.tanh((1.0-zz)*20.0) - fn.math.tanh(20.0), 4)
    Model.plasticStrain.data[:] = Model.plasticStrain.data[:] * fact.evaluate(Model.swarm)

In [23]:
Model.post_solve_functions["Plastic_strain_border"] = post_hook

## Solver, Model init

In [24]:
GEO.rcParams["advection.diffusion.method"] = "SLCN"
GEO.rcParams["initial.nonlinear.tolerance"] = 5e-2
GEO.rcParams["nonlinear.tolerance"] = 3e-2
GEO.rcParams["popcontrol.particles.per.cell.3D"] = 60
GEO.rcParams["swarm.particles.per.cell.3D"] = 60
#Model.solver.set_penalty(1.0)

In [29]:
Model.init_model()

In [ ]:
Model.run_for(nstep=1, checkpoint_interval=1)